# Material de aula - Redes Neurais e Deep Learning


## <center> Laboratório 2 - Keras: Análise de Regressão utilizando preços médios de casas </center>

In [1]:
#CÉLULA KE-LIB-01
import numpy as np
import keras as K
import tensorflow as tf
import pandas as pd
import seaborn as sns
import os
from matplotlib import pyplot as plt
%matplotlib inline
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Vamos construir uma DNN para fazer as previsões de preços.

### Definição de RNA para regressão utilizando Keras. A rede terá topologia 13-(10-10)-1 com limite de 500 epocas.
### Dataset de preços médios de casas na região de Boston, com 506 itens.

### Formato do dataset: 14 campos por linha, dos quais 13 constituem os preditores e o 14o corresponde ao valor do imóvel (saída)

### [0]  = per capita crime rate
### [1]  = proportion of land zoned for large residential lots
### [2]  = proportion of non-retail acres
### [3]  = Boolean if the town borders the Charles River (0 = no, 1 = yes)
### [4]  = air pollution metric
### [5]  = average number rooms per house
### [6]  = proportion of old houses
### [7]  = weighted distance to Boston
### [8]  = index of accessibility to highways
### [9]  = tax rate
### [10] = pupil-teacher ratio
### [11] = measure of proportion of Black residents
### [12] = percentage lower socio-economic status residents.

### O Dataset foi pre-processado da seguinte forma: o campo [3] foi convertido de [0, 1] para [-1, 1], enquanto os demais parametros foram normalizados utilizando operador Min-Max entre 0 e 1. O preço final (saída) foi dividido por 10.000 de modo a manter  os valores entre 1.0 e 5.0

### 1 - Inicialização dos parâmetros. A inicialização fixa permite que os resultados possam ser reproduzidos de forma sistemática para melhorar as análises.

In [11]:
#CÉLULA KE-LIB-02
np.random.seed(4)
tf.set_random_seed(13)

### 2 - Carregamento do dataset e separação dos vetores X e Y

In [15]:
#CÉLULA KE-LIB-03
dfBoston = pd.read_csv('boston_mm_tab.csv', header=None)
dfBoston

X = dfBoston[np.arange(0,13)]
y = dfBoston[13]

### 3 - Faça a separação dos dados em treinamento e teste

In [ ]:
#CÉLULA KE-LIB-04
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

### 4 - Monte o Modelo

In [ ]:
#CÉLULA KE-LIB-05
#Montando a rede neural
tf.logging.set_verbosity(tf.logging.ERROR) #desliga os warnings do tensorflow

#Inicializador
init = K.initializers.RandomUniform(seed=1)

#Criando o otimizador
simple_sgd = K.optimizers.SGD(lr=0.010)

#Construindo o modelo (topologia)
model = K.models.Sequential()
model.add(K.layers.Dense(units=10, input_dim=13, kernel_initializer=init, activation='tanh')) #1a camada oculta
model.add(K.layers.Dense(units=10, kernel_initializer=init, activation='tanh'))               #2a camada oculta
model.add(K.layers.Dense(units=1,  kernel_initializer=init, activation=None))               #Camada de saída

#Compilando o modelo
model.compile(loss='mean_squared_error', optimizer=simple_sgd, metrics=['mse'])

### 5 - Execute o Treinamento

In [ ]:
#CÉLULA KE-LIB-06
#Treinamento
batch_size = 8
max_epochs = 500
print("Iniciando treinamento... ")
h = model.fit(X_train, y_train, batch_size=batch_size, epochs=max_epochs, shuffle=True, verbose=1)
print("Treinamento finalizado \n")

### 6 - Faça a avaliaçao do Modelo

In [ ]:
#CÉLULA KE-LIB-07
#Treinamento
y_pred = model.predict(X_train)
y_d = np.array(y_train).reshape(-1, 1)

results = abs(y_pred - y_d) < np.abs(0.15 * y_d)
results

#Acuracidade
acc = np.sum(results) / len(results)
print("Taxa de acerto do conjunto de treinamento (%): {0:.4f}".format(acc*100) )

#Teste
y_pred = model.predict(X_test)
y_d = np.array(y_test).reshape(-1, 1)

results = abs(y_pred - y_d) < np.abs(0.15 * y_d)
results

#Acuracidade
acc = np.sum(results) / len(results)
print("Taxa de acerto do conjunto de teste (%): {0:.4f}".format(acc*100) )

### 7 - Verifique o erro medio do modelo

In [ ]:
#CÉLULA KE-LIB-08
# 5 Avaliação do modelo
eval = model.evaluate(X_train, y_train, verbose=0)
print("Erro médio do conjunto de treinamento {0:.4f}".format(eval[0]))

eval = model.evaluate(X_test, y_test, verbose=0)
print("Erro médio do conjunto de teste {0:.4f}".format(eval[0]))

### 8 - Salve o modelo em Arquivo

In [ ]:
#CÉLULA KE-LIB-9
# Salvando modelo em arquivo
print("Salvando modelo em arquivo \n")
mp = ".\\boston_model.h5"
model.save(mp)

### 9 - Faça um teste de operação com o Modelo

In [ ]:
# 7. Usando modelo (operação)
np.set_printoptions(precision=4)
unknown = np.full(shape=(1,13), fill_value=0.6, dtype=np.float32)
unknown[0][3] = -1.0 # encodando o booleano
predicted = model.predict(unknown)
print("Usando o modelo para previsão de preço médio de casa para as caracteristicas: ")
print(unknown)
print("\nO preço médio será [dolares]: ")
print(predicted * 10000)